from
- https://towardsdatascience.com/an-easy-tutorial-about-sentiment-analysis-with-deep-learning-and-keras-2bf52b9cba91  
- https://github.com/sergiovirahonda/TweetsSentimentAnalysis/blob/main/TweetsSentimentPredictions.ipynb

In [2]:
import re

from tensorflow import keras
import tensorflow as tf

from sentimental_hwglu.naive_sa import NaiveSA
from sentimental_hwglu.utils import loadIMDBdataset

from nltk.tokenize import TreebankWordDetokenizer
import gensim

from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras import regularizers
from keras.models import Sequential
from keras import layers
from keras import regularizers
from keras import backend as K
from keras.callbacks import ModelCheckpoint
from sklearn.model_selection import train_test_split

2023-09-17 17:24:54.509797: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer.so.6'; dlerror: libnvinfer.so.6: cannot open shared object file: No such file or directory
2023-09-17 17:24:54.509887: W tensorflow/stream_executor/platform/default/dso_loader.cc:55] Could not load dynamic library 'libnvinfer_plugin.so.6'; dlerror: libnvinfer_plugin.so.6: cannot open shared object file: No such file or directory
2023-09-17 17:24:54.509898: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:30] Cannot dlopen some TensorRT libraries. If you would like to use Nvidia GPU with TensorRT, please make sure the missing libraries mentioned above are installed properly.
Using TensorFlow backend.


In [3]:
def depure_data(data):
    #Removing URLs with a regular expression
    url_pattern = re.compile(r'https?://\S+|www\.\S+')
    data = url_pattern.sub(r'', data)
    # Remove Emails
    data = re.sub('\S*@\S*\s?', '', data)
    # Remove new line characters
    data = re.sub('\s+', ' ', data)
    # Remove distracting single quotes
    data = re.sub("\'", "", data)
    return data


def sent_to_words(sentences):
    for sentence in sentences:
        yield(gensim.utils.simple_preprocess(str(sentence),     deacc=True))

def detokenize(text):
    return TreebankWordDetokenizer().detokenize(text)

def cleanData(df, column):
    temp = []
    #Splitting pd.Series to list
    data_to_list = df[column].values.tolist()
    for i in range(len(data_to_list)):
        temp.append(depure_data(data_to_list[i]))
    data_words = list(sent_to_words(temp))
    data = []
    for i in range(len(data_words)):
        data.append(detokenize(data_words[i]))
    print(data[:5])
    return data

def tokenizeData(data, max_words=500, max_len=None):
    tokenizer = Tokenizer(num_words=max_words)
    tokenizer.fit_on_texts(data)
    sequences = tokenizer.texts_to_sequences(data)
    reviews = pad_sequences(sequences, maxlen=max_len)
    return reviews

def LSTM_fitAndtestModel(X, y, max_words=500, test_size=0.25, epoches=5, max_len=None):
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=test_size)
    model1 = Sequential()
    model1.add(layers.Embedding(max_words, 20)) #The embedding layer
    model1.add(layers.LSTM(15,dropout=0.5)) #Our LSTM layer
    model1.add(layers.Dense(2,activation='softmax'))


    model1.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])

    checkpoint1 = ModelCheckpoint("/data/zibaldone/projects/ai/betchelorZhanna/data/models/best_model1_{}_{}_{}.hdf5".format(max_words, test_size, str(max_len)), monitor='val_accuracy', verbose=1,save_best_only=True, mode='auto', period=1,save_weights_only=False)
    history = model1.fit(X_train, y_train, epochs=epoches,validation_data=(X_test, y_test),callbacks=[checkpoint1])
    return history

def LSTM(X, y, max_words=500, max_len=None, test_size=0.25, epoches=5):
    print(" Preparing X")
    X = tokenizeData(X, max_words, max_len=max_len)
    print(" Preparing y")
    y = tf.keras.utils.to_categorical(y, 2, dtype="float32")
    print(" training the model")
    return LSTM_fitAndtestModel(X, y, max_words, test_size, epoches, max_len=max_len)

In [4]:
df = loadIMDBdataset(filename='/data/zibaldone/projects/ai/betchelorZhanna/data/datasets/aclImdb_vclean.csv')

In [5]:
# data = cleanData(df, 'reviews')
# reviews = tokenizeData(data)
# labels = tf.keras.utils.to_categorical(df.sentiment, 2, dtype="float32")

In [6]:
# reviews

In [13]:
all_histories= {}
all_scores= {}
test_split = [100, 1000, 10000, 20000, 25000]
print(" ===========================================")
print(" ")
for t in test_split:
    print(" ===========================================")
    print(" test_split: ", t)
    scores = {}
    histories = {}
    all_scores[t] = scores
    all_histories[t] = histories
    for n in [100, 200, 300]:
        print(" ===========================================")
        print(" max_len = ", n)
        for k in range(10):
            print(" ------------------------------------------")
            print(" repetition: ", k + 1, "/10")
            print(" repetition: test_split: ", t, ", max_len: ", n)
            print(" ------------------------------------------")
            history = LSTM(df.reviews, df.sentiment, max_len=n, test_size=(len(df.sentiment) - t) / len(df.sentiment))
            h = max(history.history['val_accuracy'])
            if k == 0: 
                scores[n] = [h]
                histories[n] = [history]
            else:      
                scores[n].append(h)
                histories[n].append(history)

 
 test_split:  100
 max_len =  100
 repetition:  1 /10
 repetition: test_split:  100 , max_len:  100
 Preparing X


KeyboardInterrupt: 

In [10]:
print(all_scores_None)


{100: {}}


In [ ]:
def testNone():
    all_histories_None = {}
    all_scores_None = {}
    test_split = [100, 1000, 10000, 20000, 25000]
    print(" ===========================================")
    print(" ")
    for t in test_split:
        print(" ===========================================")
        print(" test_split: ", t)
        scores = {}
        histories = {}
        all_scores_None[t] = scores
        all_histories_None[t] = histories
        for n in [None]:
            print(" ===========================================")
            print(" max_len = ", n)
            for k in range(10):
                print(" ------------------------------------------")
                print(" repetition: ", k + 1, "/10")
                print(" repetition: test_split: ", t, ", max_len: ", n)
                print(" ------------------------------------------")
                history = LSTM(df.reviews, df.sentiment, max_len=n, test_size=(len(df.sentiment) - t) / len(df.sentiment))
                h = max(history.history['val_accuracy'])
                if k == 0: 
                    scores[n] = [h]
                    histories[n] = [history]
                else:      
                    scores[n].append(h)
                    histories[n].append(history)
    return all_histories_None, all_scores_None